# Utilização de text embedding no contexto de prompt de IA Generativa

O projeto realizado para participação no [BBDW - Generathon](https://bbdw.com.br/generathon/) consiste na criação de embeddings de texto para serem utilizadas no contexto do chatbot utilizando VertexAI

Conforme definição que consta na [documentação do Google] (https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings?hl=pt-br):

*Um embedding de texto é uma representação vetorial de texto e é usado de várias maneiras para encontrar itens semelhantes*
*Alguns casos de uso comuns para incorporações de texto incluem:*

+ Pesquisa semântica: texto de pesquisa classificado por semelhança semântica.
+ Classificação: retorna a classe de itens com atributos de texto semelhantes ao texto fornecido.
+ Clustering: itens de cluster com atributos de texto semelhantes ao texto fornecido.
+ Detecção de outlier: retorna itens em que os atributos de texto estão menos relacionados ao texto fornecido.
+ Interface de conversa: agrupa grupos de frases que podem levar a respostas semelhantes, como em um espaço de incorporação no nível da conversa.

Dessa forma, vamos criar uma representação vetorial com o conteúdo do [Manual de Crédito Rural](https://www3.bcb.gov.br/mcr). Assim, quando o usuário interagir com o chatbot, faremos a busca dessa query nos índices, passando o resultado como contexto para o modelo de chat conversacional da VertexAI.

Para armazenar os embeddings, utilizaremos o [Pinecode](https://www.pinecone.io/). No site, consta a seguinte definição sobre o que é um banco de dados vetorial:

*Um banco de dados vetorial é um tipo de banco de dados que indexa e armazena incorporações de vetores para recuperação rápida e pesquisa de similaridade*

In [ ]:
import os

import pinecone

from langchain.document_loaders import pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Pinecone

In [ ]:
pinecone.init(
    api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"]
)

In [ ]:
# Leitura do documento que será utilizado como exemplo
documento = pdf.PyPDFLoader('../docs/mcr-cap_operacoes.pdf')

In [ ]:
#Split do documento

chunk_size=500
chunk_overlap=20

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

docs = text_splitter.split_documents(documento.load())

In [ ]:
#Criação dos embeddings e armazenamento no pinecode

index_name = "bbdw"

embeddings = VertexAIEmbeddings(model_name="textembedding-gecko")

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)